In [16]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from loguru import logger
import time
import pickle
import datetime

In [18]:
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run in headless mode. (Finviz does not work when loaded headless)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

In [19]:
# Set the download directory

current_directory = os.path.abspath(os.getcwd())

prefs = {
    "download.default_directory": current_directory,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

In [41]:
def login(driver, email, password):
    driver.get("https://finviz.com/login.ashx")

    # Wait for the login page to load completely
    wait = WebDriverWait(driver, 120)
    wait.until(EC.presence_of_element_located((By.NAME, "email")))

    # Enter login credentials
    email_field = driver.find_element(By.NAME, "email")
    password_field = driver.find_element(By.NAME, "password")
    login_button = driver.find_element(By.XPATH, "//input[@value='Log in']")

    email_field.send_keys(email)  
    password_field.send_keys(password)        
    login_button.click()

    # Wait for the main page to load after login
    wait.until(EC.presence_of_element_located((By.ID, "account-dropdown")))
    
    driver.get("https://finviz.com/login.ashx")
    # Save cookies to a file
    with open("cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)

In [42]:
def load_cookies(driver):
    driver.get("https://finviz.com/login.ashx")
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)
    driver.refresh()

In [45]:
driver_path = '/usr/local/bin/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [47]:
def attempt_login(driver, email, password):
    try:
        # Check if cookies file exists
        if os.path.exists("cookies.pkl"):
            load_cookies(driver)
            logger.success("Logged in with cookies.")
        else:
            raise FileNotFoundError("Cookies file does not exist.")
    except Exception as e:
        logger.error(f"Failed to log in with cookies: {e}. Retrying with credentials.")
        login(driver, email, password)
        logger.success("Logged in with credentials.")

# Usage
attempt_login(driver, email, password)

2024-06-17 14:08:43.096 | SUCCESS  | __main__:attempt_login:6 - Logged in with cookies.


In [48]:
driver.get("https://elite.finviz.com/screener.ashx?v=152&p=i1&f=cap_0.01to,geo_usa|china|france|europe|australia|belgium|canada|chinahongkong|germany|hongkong|iceland|japan|newzealand|ireland|netherlands|norway|singapore|southkorea|sweden|taiwan|unitedarabemirates|unitedkingdom|switzerland|spain,sh_curvol_o100,sh_relvol_o1,ta_change_u&ft=4&o=sharesfloat&ar=10&c=0,1,2,5,6,25,26,27,28,29,30,84,45,50,51,68,60,61,63,64,67,65,66")

wait = WebDriverWait(driver, 50)
wait.until(EC.presence_of_element_located((By.ID, "screener-content")))
logger.info("Screener loaded.")

2024-06-17 14:09:12.943 | INFO     | __main__:<module>:5 - Screener loaded.


In [49]:
before_download = set(os.listdir(current_directory))
downloaded_files = {}
retry_no = 1

try:
    while len(downloaded_files) == 0 : 
        export_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'export.ashx')]")))
        actions = ActionChains(driver)
        actions.move_to_element(export_button).click().perform()
        logger.info("Export button clicked.")
        # Sleep to allow time for download to complete
        time.sleep(5)
        # List all files in the directory after download
        after_download = set(os.listdir(current_directory))
        downloaded_files = after_download - before_download
        
        if len(downloaded_files) >= 1:
            downloaded_file = downloaded_files.pop()
            # Rename the downloaded file with current date and time
            current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            new_file_name = f"finviz_data_{current_time}.csv"
            old_file_path = os.path.join(current_directory, downloaded_file)
            new_file_path = os.path.join(current_directory, new_file_name)
            os.rename(old_file_path, new_file_path)
            logger.info(f"File renamed to: {new_file_name}")
            logger.success(f"Files downloaded to: {current_directory}")
            break
        else:
            logger.error("Download failed or multiple files downloaded.")
            if retry_no == 6:
                print("Retried 5 times and failed.")
                break
            logger.warning(f"Retrying... [{retry_no}/5]")
            retry_no = retry_no = retry_no + 1
            
except Exception as e:
    logger.error("Failed to find the export button:", e)

# Close the browser
driver.quit()

2024-06-17 14:09:16.666 | INFO     | __main__:<module>:10 - Export button clicked.
2024-06-17 14:09:21.669 | INFO     | __main__:<module>:25 - File renamed to: finviz_data_20240617_140921.csv
2024-06-17 14:09:21.670 | SUCCESS  | __main__:<module>:26 - Files downloaded to: /Users/arianshf/Desktop/Research Code
